<a href="https://colab.research.google.com/github/GeorgeTelles/Daily_Patterns/blob/main/Daily_Patterns.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<div>
  <img src="https://raw.githubusercontent.com/GeorgeTelles/georgetelles/f69531ec6b293b5148563588a764c010015d315e/logo_clara.png" alt="logo clara" width="300" style="display: inline-block; vertical-align: top; margin-right: 10px;">
  <img src="https://raw.githubusercontent.com/GeorgeTelles/georgetelles/f69531ec6b293b5148563588a764c010015d315e/logo_dark.png" alt="logo dark" width="300" style="display: inline-block; vertical-align: top;">
</div>

# Análise da Variação Percentual Diária de Ativos Financeiros: Uma Abordagem Quantitativa

## Resumo

Este projeto em Python realiza uma análise quantitativa detalhada da variação percentual diária de ativos financeiros, com o objetivo de identificar padrões e tendências associadas aos dias da semana. Utilizando dados de preços históricos extraídos do Yahoo Finance e informações setoriais fornecidas pela biblioteca `tradingcomdados`, este estudo examina se a variação percentual dos preços de ativos segue padrões sistemáticos em diferentes dias da semana.

## Objetivo

O objetivo principal deste estudo é investigar a presença de padrões na variação percentual diária dos preços dos ativos ao longo dos dias da semana. A análise busca responder à seguinte questão de pesquisa: Existe uma tendência recorrente na variação percentual dos ativos financeiros em determinados dias da semana? Essa análise pode fornecer insights valiosos para investidores e analistas financeiros.

## Metodologia

1. **Coleta de Dados**:
   - **Fontes de Dados**: Dados históricos de preços dos ativos são obtidos através da biblioteca `yfinance`, enquanto informações setoriais são extraídas utilizando `tradingcomdados`.
   - **Ativos Analisados**: Incluem ativos listados na B3, componentes do índice Ibovespa (IBOV), e um ativo personalizado.

2. **Processamento dos Dados**:
   - **Cálculo da Variação Percentual**
   - **Tratamento de Outliers**: Outliers são identificados e removidos com base em desvios padrão para garantir a integridade dos dados.
   - **Agrupamento por Dia da Semana**: Os dados são agrupados por dia da semana para calcular estatísticas descritivas como a soma e a média das variações percentuais.

3. **Análise dos Resultados**:
   - **Resultados Individuais**: São apresentados resultados detalhados para cada ativo e dia da semana, incluindo a soma total e a média das variações percentuais.
   - **Análise Agregada**: A análise média é realizada para identificar padrões gerais na variação percentual dos preços por dia da semana.

## Resultados e Discussão

O estudo gera um DataFrame contendo as seguintes métricas:
- **Empresa**: Identificação do ativo analisado.
- **Dia_Semana**: Dia da semana correspondente.
- **Resultado Total**: Soma das variações percentuais diárias para o ativo.
- **Média por Dia**: Média das variações percentuais diárias para o ativo.

Os resultados são apresentados de forma a destacar padrões significativos e diferenças entre os dias da semana. A análise agregada fornece uma visão geral das tendências da variação percentual ao longo dos dias da semana.

## Conclusão

Através desta análise, busca-se identificar se existe um padrão consistente na variação percentual dos ativos financeiros em diferentes dias da semana, o que pode informar estratégias de investimento e decisões financeiras. O estudo contribui para a compreensão dos comportamentos de mercado e pode servir como base para futuras pesquisas em finanças quantitativas.

## Requisitos

- **Bibliotecas**: `yfinance`, `tradingcomdados`, `pandas`.
- **Instalação**: As bibliotecas necessárias podem ser instaladas via pip:

    ```bash
    pip install yfinance tradingcomdados pandas
    ```

## Contribuições

Este projeto é aberto a contribuições. Pesquisadores e praticantes são incentivados a submeter sugestões e melhorias através de issues e pull requests.

## Licença

Este projeto está licenciado sob a [MIT License](LICENSE).



#1. Importanto Bibliotecas

In [ ]:
!pip install numpy
!pip install tradingcomdados
!pip install vectorbt

In [ ]:
import yfinance as yf

from tradingcomdados import alternative_data as ad

from datetime import datetime
from dateutil import parser
import pandas as pd

import vectorbt as vbt

#2. Formando a lista de ativos

**Todos os ativos da B3**

In [ ]:
ativos = ad.get_sectors("B3").TICKER + ".SA"
lista_ativos = ativos.to_list()

**Ativos do Ibovespa (IBOV)**

In [ ]:
lista_ativos = [codigo + '.SA' for codigo in ad.index_composition('ibov', mode = 'list')]

**Ativo personalizado**

In [ ]:
lista_ativos = ["^BVSP",]

#4. Processamento dos dados

In [ ]:
empresas = []
dias_semana = []
resultados = []
medias = []

traducao_dias = {
    'Monday': 'Segunda-feira',
    'Tuesday': 'Terça-feira',
    'Wednesday': 'Quarta-feira',
    'Thursday': 'Quinta-feira',
    'Friday': 'Sexta-feira',
    'Saturday': 'Sábado',
    'Sunday': 'Domingo'
}

for ativo in lista_ativos:
    try:
        candles = yf.download(ativo, "2020-01-01")
        rates_frame = candles.copy()
        rates_frame['Dia_Semana'] = rates_frame.index.day_name()
        rates_frame['Dia_Semana_PT'] = rates_frame['Dia_Semana'].map(traducao_dias)
        rates_frame['Variacao_Porcentagem'] = ((rates_frame['Close'] - rates_frame['Open']) / rates_frame['Open']) * 100

        #Tratando os outliers:
        mean = rates_frame['Variacao_Porcentagem'].mean()
        std_dev = rates_frame['Variacao_Porcentagem'].std()
        lower_bound = mean - 3 * std_dev
        upper_bound = mean + 3 * std_dev

        # Atualizar rates_frame para remover outliers
        rates_frame = rates_frame[
            (rates_frame['Variacao_Porcentagem'] >= lower_bound) &
            (rates_frame['Variacao_Porcentagem'] <= upper_bound)
        ]

        grupo_dia_semana = rates_frame.groupby('Dia_Semana')['Variacao_Porcentagem']

        # Iterar sobre os grupos e imprimir os dados
        for nome_grupo, indice_grupo in grupo_dia_semana.groups.items():

            dados_grupo = rates_frame.loc[indice_grupo, 'Variacao_Porcentagem']
            # print(f"Dados do grupo {nome_grupo}:")
            # print(dados_grupo)

            resultado_soma = grupo_dia_semana.sum().loc[nome_grupo]  # Calcular soma para o grupo específico

            resultado_media = grupo_dia_semana.mean().loc[nome_grupo]  # Calcular média para o grupo específico

            resultado_formatado = "{:.2f}".format(resultado_soma)
            resultado_media_formatado = "{:.2f}".format(resultado_media)


            # # Verificar se há algum dado negativo
            # if any(dados_grupo < -5):
            #     print(f"Dia(s) da semana com variação negativa encontrado(s). Descartando...")
            #     continue

            # if resultado_soma < 90:
            #     print(f"Soma dos resultados é inferior a mèdia. Descartando...")
            #     continue

            empresas.append(ativo)
            dias_semana.append(traducao_dias.get(nome_grupo, nome_grupo))
            resultados.append(resultado_formatado)
            medias.append(resultado_media_formatado)


    except Exception as e:
        print(f"Erro ao processar o ativo {ativo}: {e}")

df_resultados = pd.DataFrame({
    'Empresa': empresas,
    'Dia_Semana': dias_semana,
    'Resultado Total': resultados,
    'Media por dia': medias
})
df_resultados['Resultado Total'] = pd.to_numeric(df_resultados['Resultado Total'])
df_resultados['Media por dia'] = pd.to_numeric(df_resultados['Media por dia'])

#5. Resultado de cada empresa e cada dia da semana

In [ ]:
df_resultados_separado = df_resultados.sort_values(by='Resultado Total', ascending=False)
df_resultados_separado

#6. Resultado por dia da semana tirando a média do mercado

In [ ]:
df_resultados.groupby("Dia_Semana")[["Resultado Total","Media por dia"]].mean().sort_values(by='Resultado Total', ascending=False)